In [19]:
from flask import Flask, request, jsonify
import psycopg2 as pg
import os
from PIL import Image
import json
import base64

In [20]:
conn = pg.connect(host="127.0.0.1", dbname="youreco", user="postgres", password="postgres", port=5432)
cur = conn.cursor()

In [21]:
section_list = [(1, 'D8mq5L9R0GM', 0, 300, 385002, [8493, 8517, 8559, 8629]), (2, 'D8mq5L9R0GM', 300, 600, 445726, [13784, 21765, 21250, 21284]), (3, 'D8mq5L9R0GM', 600, 918, 119561, [9073, 11887, 18761, 24267])]

In [22]:
section_list

[(1, 'D8mq5L9R0GM', 0, 300, 385002, [8493, 8517, 8559, 8629]),
 (2, 'D8mq5L9R0GM', 300, 600, 445726, [13784, 21765, 21250, 21284]),
 (3, 'D8mq5L9R0GM', 600, 918, 119561, [9073, 11887, 18761, 24267])]

## Get Full Recommendation List(with Item Code)

In [23]:
def get_recommendation(cateID):
    # communicate with rec server
    return [[8493, 8517, 8559, 8629],[8493, 8517, 8559, 8629],[8493, 8517, 8559, 8629]]

## replace itemcode table to itemdata sheet

In [32]:
#get iteminfo list from itemid
def item_id_to_info(itemid):
    cur.execute("select * from iteminfo where itemid = {0};".format(itemid))
    res = cur.fetchone()
    item_name = res[1]
    item_category = res[2]
    item_price = res[3]
    image_filename = res[7]
    item_link = "https://www.naver.com"
    return [item_name, item_price, image_filename, item_link]

In [33]:
#create itemdata sheet
def create_section_object(start_time, end_time, category):
    rec_dict = {
        "start_time":start_time, 
        "end_time":end_time, 
        "category": category, 
        "sim_item":[], 
        "rel_item_01" : [], 
        "rel_item_02" : [], 
        "rel_item_03" : []
    }
    
    return rec_dict

In [34]:
#full sheet that send to frontend page
new_section_list = []

In [35]:
for section in section_list:
    new_section = create_section_object(section[2], section[3], section[4]);
    rel_item_list = get_recommendation(new_section["category"])
    rel_item_01_code = rel_item_list[0]
    rel_item_02_code = rel_item_list[1]
    rel_item_03_code = rel_item_list[2]
    
    #sim_item
    for each_item in section[5]:
        item = item_id_to_info(each_item)
        new_section["sim_item"].append(item)
    
    #rel_item_01
    for each_item in rel_item_01_code:
        item = item_id_to_info(each_item)
        new_section["rel_item_01"].append(item)
    
    #rel_item_02
    for each_item in rel_item_02_code:
        item = item_id_to_info(each_item)
        new_section["rel_item_02"].append(item)
    
    #rel_item_03
    for each_item in rel_item_03_code:
        item = item_id_to_info(each_item)
        new_section["rel_item_03"].append(item)
        
        
    new_section_list.append(new_section)

In [40]:
new_section_list

[{'start_time': 0,
  'end_time': 300,
  'category': 385002,
  'sim_item': [['레드퍼피 반려견 잉클 원 방석, 부비베어',
    13600,
    'c333e61ace13',
    'https://www.naver.com'],
   ['크림펫 반려동물 극세사 마약 방석, 핑크', 16680, 'a550afe1f6b4', 'https://www.naver.com'],
   ['레드퍼피 반려견 스넉워머 원 방석, 아이보리',
    20900,
    '6a84193d7833',
    'https://www.naver.com'],
   ['포글 반려동물 사계절 강아지 방석 허그 쿠션, 클리어민트',
    36900,
    'f80b752a10c7',
    'https://www.naver.com']],
  'rel_item_01': [['레드퍼피 반려견 잉클 원 방석, 부비베어',
    13600,
    'c333e61ace13',
    'https://www.naver.com'],
   ['크림펫 반려동물 극세사 마약 방석, 핑크', 16680, 'a550afe1f6b4', 'https://www.naver.com'],
   ['레드퍼피 반려견 스넉워머 원 방석, 아이보리',
    20900,
    '6a84193d7833',
    'https://www.naver.com'],
   ['포글 반려동물 사계절 강아지 방석 허그 쿠션, 클리어민트',
    36900,
    'f80b752a10c7',
    'https://www.naver.com']],
  'rel_item_02': [['레드퍼피 반려견 잉클 원 방석, 부비베어',
    13600,
    'c333e61ace13',
    'https://www.naver.com'],
   ['크림펫 반려동물 극세사 마약 방석, 핑크', 16680, 'a550afe1f6b4', 'https://www.naver.com'],


In [41]:
json_val = json.dumps(new_section_list)

In [42]:
json_val

'[{"start_time": 0, "end_time": 300, "category": 385002, "sim_item": [["\\ub808\\ub4dc\\ud37c\\ud53c \\ubc18\\ub824\\uacac \\uc789\\ud074 \\uc6d0 \\ubc29\\uc11d, \\ubd80\\ube44\\ubca0\\uc5b4", 13600, "c333e61ace13", "https://www.naver.com"], ["\\ud06c\\ub9bc\\ud3ab \\ubc18\\ub824\\ub3d9\\ubb3c \\uadf9\\uc138\\uc0ac \\ub9c8\\uc57d \\ubc29\\uc11d, \\ud551\\ud06c", 16680, "a550afe1f6b4", "https://www.naver.com"], ["\\ub808\\ub4dc\\ud37c\\ud53c \\ubc18\\ub824\\uacac \\uc2a4\\ub109\\uc6cc\\uba38 \\uc6d0 \\ubc29\\uc11d, \\uc544\\uc774\\ubcf4\\ub9ac", 20900, "6a84193d7833", "https://www.naver.com"], ["\\ud3ec\\uae00 \\ubc18\\ub824\\ub3d9\\ubb3c \\uc0ac\\uacc4\\uc808 \\uac15\\uc544\\uc9c0 \\ubc29\\uc11d \\ud5c8\\uadf8 \\ucfe0\\uc158, \\ud074\\ub9ac\\uc5b4\\ubbfc\\ud2b8", 36900, "f80b752a10c7", "https://www.naver.com"]], "rel_item_01": [["\\ub808\\ub4dc\\ud37c\\ud53c \\ubc18\\ub824\\uacac \\uc789\\ud074 \\uc6d0 \\ubc29\\uc11d, \\ubd80\\ube44\\ubca0\\uc5b4", 13600, "c333e61ace13", "https://www.n